## Erkunde K-Means-Clustering mit R und den Prinzipien von Tidy Data.

### [**Quiz vor der Lektion**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

In dieser Lektion lernst du, wie du mit dem Tidymodels-Paket und anderen Paketen aus dem R-Ökosystem (wir nennen sie Freunde 🧑‍🤝‍🧑) sowie dem nigerianischen Musikdatensatz, den du zuvor importiert hast, Cluster erstellen kannst. Wir behandeln die Grundlagen von K-Means für das Clustering. Denke daran, dass es, wie du in der vorherigen Lektion gelernt hast, viele Möglichkeiten gibt, mit Clustern zu arbeiten, und die Methode, die du wählst, von deinen Daten abhängt. Wir werden K-Means ausprobieren, da es die am häufigsten verwendete Clustering-Technik ist. Los geht's!

Begriffe, die du kennenlernen wirst:

-   Silhouettenbewertung

-   Elbow-Methode

-   Trägheit

-   Varianz

### **Einführung**

[K-Means-Clustering](https://wikipedia.org/wiki/K-means_clustering) ist eine Methode, die aus dem Bereich der Signalverarbeitung stammt. Sie wird verwendet, um Gruppen von Daten basierend auf Ähnlichkeiten ihrer Merkmale in `k Cluster` zu unterteilen und zu partitionieren.

Die Cluster können als [Voronoi-Diagramme](https://wikipedia.org/wiki/Voronoi_diagram) visualisiert werden, die einen Punkt (oder 'Seed') und dessen entsprechende Region enthalten.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografik von Jen Looper</figcaption>


Die Schritte des K-Means-Clustering sind wie folgt:

1.  Der Datenwissenschaftler beginnt damit, die gewünschte Anzahl von Clustern festzulegen, die erstellt werden sollen.

2.  Anschließend wählt der Algorithmus zufällig K Beobachtungen aus dem Datensatz aus, die als anfängliche Zentren der Cluster (d. h. Zentroiden) dienen.

3.  Danach wird jede der verbleibenden Beobachtungen ihrem nächstgelegenen Zentroiden zugewiesen.

4.  Anschließend werden die neuen Mittelwerte jedes Clusters berechnet und der Zentroid wird zum Mittelwert verschoben.

5.  Nun, da die Zentren neu berechnet wurden, wird jede Beobachtung erneut überprüft, um festzustellen, ob sie möglicherweise näher an einem anderen Cluster liegt. Alle Objekte werden erneut unter Verwendung der aktualisierten Cluster-Mittelwerte zugewiesen. Die Schritte der Clusterzuweisung und der Aktualisierung der Zentroiden werden iterativ wiederholt, bis sich die Clusterzuweisungen nicht mehr ändern (d. h. wenn Konvergenz erreicht ist). Typischerweise endet der Algorithmus, wenn jede neue Iteration zu einer vernachlässigbaren Bewegung der Zentroiden führt und die Cluster statisch werden.

<div>

> Beachte, dass aufgrund der Zufälligkeit der anfänglichen k Beobachtungen, die als Startzentroiden verwendet werden, jedes Mal, wenn wir das Verfahren anwenden, leicht unterschiedliche Ergebnisse erzielt werden können. Aus diesem Grund verwenden die meisten Algorithmen mehrere *zufällige Starts* und wählen die Iteration mit dem niedrigsten WCSS. Daher wird dringend empfohlen, K-Means immer mit mehreren Werten von *nstart* auszuführen, um ein *unerwünschtes lokales Optimum* zu vermeiden.

</div>

Diese kurze Animation, die die [Illustrationen](https://github.com/allisonhorst/stats-illustrations) von Allison Horst verwendet, erklärt den Clustering-Prozess:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Illustration von @allison_horst</figcaption>



Eine grundlegende Frage, die beim Clustering aufkommt, ist: Wie weißt du, in wie viele Cluster du deine Daten aufteilen sollst? Ein Nachteil der Verwendung von K-Means ist, dass du `k`, also die Anzahl der `Zentroiden`, festlegen musst. Glücklicherweise hilft die `Elbow-Methode`, einen guten Ausgangswert für `k` zu schätzen. Du wirst es gleich ausprobieren.

### 

**Voraussetzung**

Wir machen genau dort weiter, wo wir in der [vorherigen Lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) aufgehört haben, in der wir den Datensatz analysiert, viele Visualisierungen erstellt und den Datensatz auf interessante Beobachtungen gefiltert haben. Schau sie dir unbedingt an!

Wir benötigen einige Pakete, um dieses Modul abzuschließen. Du kannst sie wie folgt installieren: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativ überprüft das folgende Skript, ob du die für dieses Modul erforderlichen Pakete hast, und installiert sie für dich, falls einige fehlen.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Los geht's!

## 1. Ein Tanz mit Daten: Eingrenzen auf die 3 beliebtesten Musikgenres

Dies ist eine Zusammenfassung dessen, was wir in der vorherigen Lektion gemacht haben. Lass uns ein paar Daten analysieren!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Das lief gut!

## 2. Mehr Datenexploration.

Wie sauber sind diese Daten? Lassen Sie uns Ausreißer mithilfe von Boxplots überprüfen. Wir konzentrieren uns auf numerische Spalten mit weniger Ausreißern (auch wenn Sie die Ausreißer bereinigen könnten). Boxplots können den Wertebereich der Daten zeigen und helfen, auszuwählen, welche Spalten verwendet werden sollen. Beachten Sie, dass Boxplots keine Varianz darstellen, ein wichtiger Aspekt für gut clustbare Daten. Weitere Informationen finden Sie in [dieser Diskussion](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Boxplots](https://de.wikipedia.org/wiki/Box-Plot) werden verwendet, um die Verteilung von `numerischen` Daten grafisch darzustellen. Beginnen wir also damit, alle numerischen Spalten zusammen mit den beliebten Musikgenres *auszuwählen*.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Sehen Sie, wie der Auswahlhelfer `where` das so einfach macht 💁? Entdecken Sie weitere solche Funktionen [hier](https://tidyselect.r-lib.org/).

Da wir für jede numerische Eigenschaft ein Boxplot erstellen werden und Schleifen vermeiden möchten, formatieren wir unsere Daten in ein *längeres* Format um. Dadurch können wir `facets` nutzen – Unterdiagramme, die jeweils einen Teil der Daten anzeigen.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Viel länger! Jetzt ist es Zeit für einige `ggplots`! Welches `geom` werden wir verwenden?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Jetzt können wir sehen, dass diese Daten etwas unruhig sind: Wenn man jede Spalte als Boxplot betrachtet, erkennt man Ausreißer. Man könnte den Datensatz durchgehen und diese Ausreißer entfernen, aber das würde die Daten ziemlich minimal machen.

Für den Moment wählen wir aus, welche Spalten wir für unsere Cluster-Übung verwenden werden. Lassen Sie uns die numerischen Spalten mit ähnlichen Bereichen auswählen. Wir könnten `artist_top_genre` als numerisch codieren, aber wir lassen es vorerst weg.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Berechnung von k-means Clustering in R

Wir können k-means in R mit der integrierten Funktion `kmeans` berechnen, siehe `help("kmeans()")`. Die Funktion `kmeans()` akzeptiert als Hauptargument einen Data Frame mit ausschließlich numerischen Spalten.

Der erste Schritt bei der Verwendung von k-means Clustering besteht darin, die Anzahl der Cluster (k) festzulegen, die in der endgültigen Lösung erzeugt werden sollen. Wir wissen, dass es 3 Musikgenres gibt, die wir aus dem Datensatz herausgearbeitet haben, also probieren wir es mit 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Das kmeans-Objekt enthält mehrere Informationen, die in `help("kmeans()")` gut erklärt werden. Für den Moment konzentrieren wir uns auf einige davon. Wir sehen, dass die Daten in 3 Cluster mit den Größen 65, 110, 111 gruppiert wurden. Die Ausgabe enthält außerdem die Clusterzentren (Mittelwerte) für die 3 Gruppen über die 5 Variablen.

Der Clustering-Vektor ist die Clusterzuweisung für jede Beobachtung. Nutzen wir die Funktion `augment`, um die Clusterzuweisung dem ursprünglichen Datensatz hinzuzufügen.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfekt, wir haben gerade unseren Datensatz in 3 Gruppen aufgeteilt. Aber wie gut ist unser Clustering 🤷? Schauen wir uns den `Silhouette-Score` an.

### **Silhouette-Score**

[Silhouette-Analyse](https://en.wikipedia.org/wiki/Silhouette_(clustering)) kann verwendet werden, um die Trennungsdistanz zwischen den resultierenden Clustern zu untersuchen. Dieser Score variiert zwischen -1 und 1. Wenn der Score nahe bei 1 liegt, ist das Cluster dicht und gut von anderen Clustern getrennt. Ein Wert nahe 0 repräsentiert sich überlappende Cluster, bei denen die Datenpunkte sehr nah an der Entscheidungsgrenze der benachbarten Cluster liegen. [Quelle](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Die Methode des durchschnittlichen Silhouette-Scores berechnet den durchschnittlichen Silhouette-Wert der Beobachtungen für verschiedene Werte von *k*. Ein hoher durchschnittlicher Silhouette-Score deutet auf ein gutes Clustering hin.

Die `silhouette`-Funktion im Cluster-Paket wird verwendet, um die durchschnittliche Silhouette-Breite zu berechnen.

> Der Silhouette-Score kann mit jeder [Distanz](https://en.wikipedia.org/wiki/Distance "Distance")-Metrik berechnet werden, wie z. B. der [euklidischen Distanz](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") oder der [Manhattan-Distanz](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), die wir in der [vorherigen Lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) besprochen haben.


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Unser Score ist **0,549**, also genau in der Mitte. Das deutet darauf hin, dass unsere Daten nicht besonders gut für diese Art der Clusterbildung geeignet sind. Schauen wir, ob wir diese Vermutung visuell bestätigen können. Das [factoextra-Paket](https://rpkgs.datanovia.com/factoextra/index.html) stellt Funktionen (`fviz_cluster()`) zur Verfügung, um Cluster visuell darzustellen.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Die Überlappung der Cluster zeigt, dass unsere Daten für diese Art der Clusterbildung nicht besonders gut geeignet sind, aber lassen Sie uns trotzdem weitermachen.

## 4. Bestimmung der optimalen Anzahl von Clustern

Eine grundlegende Frage, die bei der K-Means-Clusterbildung häufig auftritt, lautet: Ohne bekannte Klassenlabels, wie wissen wir, in wie viele Cluster die Daten unterteilt werden sollen?

Eine Möglichkeit, dies herauszufinden, besteht darin, eine Datenprobe zu verwenden, um `eine Reihe von Clustering-Modellen` mit einer steigenden Anzahl von Clustern zu erstellen (z. B. von 1 bis 10) und Clustering-Metriken wie den **Silhouette-Score** zu bewerten.

Lassen Sie uns die optimale Anzahl von Clustern bestimmen, indem wir den Clustering-Algorithmus für verschiedene Werte von *k* berechnen und die **Summe der Quadrate innerhalb der Cluster** (WCSS) bewerten. Die gesamte Summe der Quadrate innerhalb der Cluster (WCSS) misst die Kompaktheit der Clusterbildung, und wir möchten, dass sie so klein wie möglich ist. Niedrigere Werte bedeuten, dass die Datenpunkte näher beieinander liegen.

Lassen Sie uns die Auswirkungen verschiedener Werte von `k`, von 1 bis 10, auf diese Clusterbildung untersuchen.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nun, da wir die gesamte innerhalb-Cluster-Summe der Quadrate (tot.withinss) für jeden Clustering-Algorithmus mit Zentrum *k* haben, verwenden wir die [Elbow-Methode](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), um die optimale Anzahl von Clustern zu finden. Die Methode besteht darin, die WCSS als Funktion der Anzahl der Cluster zu plotten und den [Knick der Kurve](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Knick der Kurve") als die Anzahl der zu verwendenden Cluster auszuwählen.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Die Grafik zeigt eine deutliche Reduktion des WCSS (also eine größere *Kompaktheit*), wenn die Anzahl der Cluster von eins auf zwei steigt, sowie eine weitere merkliche Reduktion von zwei auf drei Cluster. Danach wird die Reduktion weniger ausgeprägt, was zu einem `Knick` 💪 im Diagramm bei etwa drei Clustern führt. Dies ist ein guter Hinweis darauf, dass es zwei bis drei einigermaßen gut getrennte Cluster von Datenpunkten gibt.

Wir können nun das Clustering-Modell extrahieren, bei dem `k = 3` ist:

> `pull()`: wird verwendet, um eine einzelne Spalte zu extrahieren
>
> `pluck()`: wird verwendet, um Datenstrukturen wie Listen zu indexieren


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Super! Lassen Sie uns die erhaltenen Cluster visualisieren. Lust auf etwas Interaktivität mit `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Vielleicht hätten wir erwartet, dass jeder Cluster (repräsentiert durch verschiedene Farben) unterschiedliche Genres (repräsentiert durch verschiedene Formen) aufweist.

Werfen wir einen Blick auf die Genauigkeit des Modells.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Die Genauigkeit dieses Modells ist nicht schlecht, aber auch nicht besonders gut. Es könnte daran liegen, dass die Daten sich nicht gut für K-Means-Clustering eignen. Die Daten sind zu unausgewogen, zu wenig korreliert, und es gibt zu viel Varianz zwischen den Spaltenwerten, um gut zu clustern. Tatsächlich werden die gebildeten Cluster wahrscheinlich stark von den drei oben definierten Genre-Kategorien beeinflusst oder verzerrt.

Nichtsdestotrotz war das ein ziemlich lehrreicher Prozess!

In der Dokumentation von Scikit-learn kann man sehen, dass ein Modell wie dieses, bei dem die Cluster nicht klar abgegrenzt sind, ein 'Varianz'-Problem hat:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografik von Scikit-learn</figcaption>



## **Varianz**

Varianz wird definiert als "der Durchschnitt der quadrierten Abweichungen vom Mittelwert" [Quelle](https://www.mathsisfun.com/data/standard-deviation.html). Im Kontext dieses Clustering-Problems bezieht sich dies darauf, dass die Zahlen in unserem Datensatz dazu neigen, sich zu stark vom Mittelwert zu entfernen.

✅ Dies ist ein großartiger Moment, um über alle Möglichkeiten nachzudenken, wie man dieses Problem beheben könnte. Die Daten noch etwas anpassen? Andere Spalten verwenden? Einen anderen Algorithmus ausprobieren? Tipp: Versuchen Sie, [Ihre Daten zu skalieren](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/), um sie zu normalisieren, und testen Sie andere Spalten.

> Probieren Sie diesen '[Varianzrechner](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' aus, um das Konzept besser zu verstehen.

------------------------------------------------------------------------

## **🚀Herausforderung**

Verbringen Sie etwas Zeit mit diesem Notebook und passen Sie die Parameter an. Können Sie die Genauigkeit des Modells verbessern, indem Sie die Daten weiter bereinigen (z. B. Ausreißer entfernen)? Sie können Gewichte verwenden, um bestimmten Datenproben mehr Gewicht zu geben. Was können Sie sonst noch tun, um bessere Cluster zu erstellen?

Tipp: Versuchen Sie, Ihre Daten zu skalieren. Im Notebook gibt es auskommentierten Code, der eine Standard-Skalierung hinzufügt, um die Daten-Spalten in Bezug auf den Wertebereich einander ähnlicher zu machen. Sie werden feststellen, dass der Silhouetten-Score zwar sinkt, aber der 'Knick' im Elbogen-Diagramm geglättet wird. Das liegt daran, dass unskalierte Daten es Daten mit weniger Varianz ermöglichen, mehr Gewicht zu tragen. Lesen Sie mehr über dieses Problem [hier](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz nach der Vorlesung**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Überblick & Selbststudium**

-   Schauen Sie sich einen K-Means-Simulator [wie diesen hier](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/) an. Mit diesem Tool können Sie Beispieldatenpunkte visualisieren und deren Zentroiden bestimmen. Sie können die Zufälligkeit der Daten, die Anzahl der Cluster und die Anzahl der Zentroiden bearbeiten. Hilft Ihnen das, eine Vorstellung davon zu bekommen, wie die Daten gruppiert werden können?

-   Werfen Sie auch einen Blick auf [dieses Handout zu K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) von Stanford.

Möchten Sie Ihre neu erworbenen Clustering-Fähigkeiten an Datensätzen ausprobieren, die sich gut für K-Means-Clustering eignen? Schauen Sie sich Folgendes an:

-   [Train and Evaluate Clustering Models](https://rpubs.com/eR_ic/clustering) mit Tidymodels und Co.

-   [K-means Cluster Analysis](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering with tidy data principles](https://www.tidymodels.org/learn/statistics/k-means/)

## **Aufgabe**

[Probieren Sie verschiedene Clustering-Methoden aus](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## EIN GROSSES DANKESCHÖN AN:

[Jen Looper](https://www.twitter.com/jenlooper) für die Erstellung der ursprünglichen Python-Version dieses Moduls ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) für die Erstellung der großartigen Illustrationen, die R einladender und ansprechender machen. Weitere Illustrationen finden Sie in ihrer [Galerie](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Viel Spaß beim Lernen,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>



---

**Haftungsausschluss**:  
Dieses Dokument wurde mithilfe des KI-Übersetzungsdienstes [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
